In [3]:
from math import log

In [62]:
s1='RT @MattMonge: If you can make time to tell someone what they did wrong, you can make time to tell someone what they did right. #leadership...'
'You can make time to tell someone what they did wrong, you can make time to tell someone what they did right. Leadership...'

'You can make time to tell someone what they did wrong, you can make time to tell someone what they did right. Leadership...'

In [8]:
import tweepy
import json
from ttp import ttp
from math import log
from unidecode import unidecode
import HTMLParser
from nltk import sent_tokenize
from keys import keys
from hashtags import *

In [85]:
def strip_tags(text):
    text = text.replace('\n', ' ')

    # get rid of period sometimes placed before @username in the beginning
    if text[0] == '.':
        text = text[1:]

    if text[-3:] == '...':
        text = text[:-3]

    tokens = text.split()
    for ix, word in enumerate(tokens):
        if word == 'RT':
            tokens.pop(ix)
            try:
                tokens.pop(ix)  # try to pop what was behind the RT
            except:
                pass
    text = ' '.join(tokens)

    # remove junk at the end that is just a stream of hashtags and @'s'
    tokens = text.lower().split()
    more_slop = True
    while more_slop:
        first_char_last_token = tokens[-1][0]
        last_char_last_token = tokens[-1][-1]
        if tokens[-1].startswith('http'):
            tokens.pop()
            text = ' '.join(tokens)
        elif first_char_last_token in '.#@([' and last_char_last_token not in '.!?':
            tokens.pop()
            text = ' '.join(tokens)
        elif tokens[-1] in ['via', 'on', 'with']:
            tokens.pop()
            text = ' '.join(tokens)
        else:
            more_slop = False

    p = ttp.Parser()
    result = p.parse(text)

    for i in result.users:
        text = text.replace('@'+i, i.capitalize())

    # remove links
    for i in result.urls:
        text = text.replace(i+', ', '')
        text = text.replace(i+' ', '')
        text = text.replace(' '+i, '')
        text = text.replace(i, '') # where tweet consists only of the link

    # remove last word if now it is via after stripping an @-name
    tokens = text.split()
    last_word = tokens[-1]
    slop = ['via', 'on', 'with']
    if last_word in slop or len(last_word) < 2:
        tokens.pop()
        text = ' '.join(tokens)

    # remove junk at front of tweet
    tokens = text.split()
    more_slop = True
    while more_slop:
        first_char_first_token = tokens[0][0]
        if first_char_first_token in ['#', '@']:
            tokens.pop(0)
            text = ' '.join(tokens)
        else:
            more_slop = False

    # remove first word if it is now colon or ends in colon
    tokens = text.split()
    first_word = tokens[0]
    if first_word[-1] == ':':
        tokens.pop(0)
        text = ' '.join(tokens)

    result = p.parse(text)

    # unravel hashtags and remove hashmark
    for i in result.tags:
        unraveled = infer_spaces(i)
        text = text.replace('#'+i, unraveled)

    text = text.strip()

    if text[-1] in ':-|':
        text = text[:-1]
    if text[-1] in ['=>', ' -', '->']:
        text = text[:-2] + '.'
    if text[-1] not in '.?!':
        text = text + '.'

    return sentence_case(text.strip())


In [86]:
print s1
print strip_tags(s1)

RT @MattMonge: If you can make time to tell someone what they did wrong, you can make time to tell someone what they did right. #leadership...
If you can make time to tell someone what they did wrong, you can make time to tell someone what they did right.


# fix missing if and fix leadership at the end 

In [84]:
text = s1
if text[-3] == '...':
    text = text[:-3]
text

'RT @MattMonge: If you can make time to tell someone what they did wrong, you can make time to tell someone what they did right. #leadership...'